<a href="https://colab.research.google.com/github/Meerschwein/Automating-SE/blob/main/Milestone3-v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###############################################################################
#  0.  INSTALL & IMPORTS   ####################################################
###############################################################################
!pip -q uninstall -y peft sentence-transformers
!pip -q install transformers==4.40.2 datasets==2.18.0 accelerate scikit-learn -U

from pathlib import Path
import urllib.request, zipfile, json, random, torch, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModel,                     #  M1
    AutoModelForSequenceClassification,           #  M2
    AutoModelForTokenClassification,              #  M3
    DataCollatorWithPadding,
    DataCollatorForTokenClassification,
    TrainingArguments, Trainer)
from tqdm.auto import tqdm

seed = 42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
###############################################################################
#  1.  DOWNLOAD                                                               #
###############################################################################
zip_url  = "https://raw.githubusercontent.com/Meerschwein/Automating-SE/refs/heads/main/Big-Vul-dataset.zip"
zip_path = Path("Big-Vul-dataset.zip")
data_dir = Path("Big-Vul-dataset")
if not zip_path.exists():
    urllib.request.urlretrieve(zip_url, zip_path)
if not data_dir.exists():
    with zipfile.ZipFile(zip_path) as z: z.extractall(data_dir)

json_path = data_dir / "data.json"
assert json_path.exists()

In [3]:
###############################################################################
#  2.  LOAD                                                                    #
###############################################################################
try:
    df = pd.read_json(json_path, lines=True, dtype={"vul": "int8"})
except ValueError:                                           # JSON array
    df = pd.read_json(json_path, dtype={"vul": "int8"})

df = (df.rename(columns={"vul": "label"})          # keep flaw_line_no for M3
        .dropna(subset=["code", "label"])
        .drop_duplicates("code")
        .reset_index(drop=True))

print(f"Loaded {len(df):,} functions "
      f"(vuln {df.label.sum():,} / benign {(df.label==0).sum():,})")

Loaded 186,275 functions (vuln 8,782 / benign 177,493)


In [4]:
###############################################################################
#  3-A.  M1 – GET *EMBEDDINGS ONLY*                                           #
###############################################################################
model_name = "microsoft/codebert-base"
tok        = AutoTokenizer.from_pretrained(model_name)
embedder   = AutoModel.from_pretrained(model_name).eval().cuda()   # no head

def code2emb(code: str):
    """Return [CLS] 768-d embedding for a single C/C++ function."""
    inputs = tok(code, return_tensors="pt", truncation=True,
                 max_length=512).to(embedder.device)
    with torch.no_grad():
        last_hidden = embedder(**inputs).last_hidden_state
    return last_hidden[:,0,:].squeeze().cpu()      # [CLS] vector (1,768)

# ▶️ DEMO – first function
print("Embedding shape:", code2emb(df.code.iloc[0]).shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Embedding shape: torch.Size([768])


In [5]:
###############################################################################
#  4.  SPLIT TO TRAIN / VAL / TEST                                            #
###############################################################################
train_df, test_df = train_test_split(df, test_size=0.15,
                                     stratify=df.label, random_state=seed)
train_df,  val_df = train_test_split(train_df, test_size=0.10,
                                     stratify=train_df.label, random_state=seed)

def to_hf(dframe): return Dataset.from_pandas(dframe, preserve_index=False)
train_func, val_func, test_func = map(to_hf, (train_df, val_df, test_df))

In [6]:
###############################################################################
#  5.  COMMON TOKENISER & COLLATOR                                            #
###############################################################################
def tok_func(batch):
    return tok(batch["code"], truncation=True,
               max_length=512, padding="max_length")

train_func = train_func.map(tok_func, batched=True, remove_columns=["code"])
val_func   = val_func  .map(tok_func, batched=True, remove_columns=["code"])
test_func  = test_func .map(tok_func, batched=True, remove_columns=["code"])

func_collator = DataCollatorWithPadding(tok)

Map:   0%|          | 0/142499 [00:00<?, ? examples/s]

Map:   0%|          | 0/15834 [00:00<?, ? examples/s]

Map:   0%|          | 0/27942 [00:00<?, ? examples/s]

In [7]:
###############################################################################
#  6.  M2 – FUNCTION-LEVEL CLASSIFIER                                         #
###############################################################################
func_model = AutoModelForSequenceClassification.from_pretrained(
                model_name, num_labels=2).cuda()

# train *all* layers for best quality
for p in func_model.base_model.parameters():
    p.requires_grad = True

neg, pos = train_df.label.value_counts().sort_index()
class_weights = torch.tensor([1.0, (neg/pos)**0.5], dtype=torch.float).cuda()

class WTrainer(Trainer):
    def __init__(self, class_weights=None, **kw):
        super().__init__(**kw); self.w = class_weights
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs["labels"]; outputs = model(**inputs)
        loss = torch.nn.functional.cross_entropy(outputs.logits, labels,
                                                 weight=self.w)
        return (loss, outputs) if return_outputs else loss

def metrics(eval_pred):
    logits, y = eval_pred; p = np.argmax(logits, -1)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y, p, average="binary", zero_division=0, pos_label=1)
    return {"accuracy": accuracy_score(y,p),
            "precision":prec, "recall":rec, "f1":f1}

args = TrainingArguments(
    "func-results", fp16=True, per_device_train_batch_size=64,
    per_device_eval_batch_size=64, num_train_epochs=3,
    evaluation_strategy="epoch", save_strategy="epoch",
    learning_rate=2e-5, weight_decay=0.01, load_best_model_at_end=True,
    metric_for_best_model="f1", seed=seed, report_to="none")

func_trainer = WTrainer(
    model=func_model, args=args,
    train_dataset=train_func, eval_dataset=val_func,
    tokenizer=tok, data_collator=func_collator,
    compute_metrics=metrics, class_weights=class_weights)

print("===  M2  training (function level)  ===")
func_trainer.train()

print("\n===  Test metrics (function level) ===")
print(func_trainer.evaluate(test_func))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


===  M2  training (function level)  ===


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.178300,0.144350,0.979285,0.787380,0.768407,0.777778
2,0.147200,0.133766,0.984716,0.885496,0.776439,0.827389
3,0.126400,0.140403,0.985095,0.868687,0.805890,0.836111



===  Test metrics (function level) ===


{'eval_loss': 0.16919633746147156, 'eval_accuracy': 0.9836446925774819, 'eval_precision': 0.8571428571428571, 'eval_recall': 0.7835990888382688, 'eval_f1': 0.8187227290757636, 'eval_runtime': 65.5072, 'eval_samples_per_second': 426.548, 'eval_steps_per_second': 6.671, 'epoch': 3.0}


In [8]:
#  -- after M2 training  --
func_trainer.save_model("func-classifier")     # model + config
tok.save_pretrained("func-classifier")         # tokenizer (same for all stages)

('func-classifier/tokenizer_config.json',
 'func-classifier/special_tokens_map.json',
 'func-classifier/vocab.json',
 'func-classifier/merges.txt',
 'func-classifier/added_tokens.json',
 'func-classifier/tokenizer.json')

In [9]:
###############################################################################
#  7.  M3 – LINE-LEVEL (TOKEN) CLASSIFIER                                     #
###############################################################################
# --------------------------------------------------------------------------- #
# 7-A. build token-wise labels                                                #
# --------------------------------------------------------------------------- #
def add_token_labels(example):
    """
    Return tokenised code + `labels` (1=vulnerable-line, 0=clean-line, -100=pad)
    so that `labels` has exactly the same length as input_ids (512 after padding).
    """
    code        = example["code"]
    vuln_lines  = set(example["flaw_line_no"])          # [] if benign

    # --- run the tokenizer with padding *now* --------------------------------
    enc = tok(
        code,
        return_offsets_mapping=True,
        truncation=True,
        max_length=512,
        padding="max_length",
    )

    # --- map every token to its source-code line -----------------------------
    labels = np.full(len(enc["input_ids"]), -100, dtype=np.int8)   # pad value
    line_start = [0] + [i + 1 for i, c in enumerate(code) if c == "\n"]

    for idx, (start, _) in enumerate(enc["offset_mapping"]):
        if start == 0 and idx == 0:          # [CLS] token → keep -100
            continue
        # line numbers are 1-based
        line_no = 1 + sum(start >= ls for ls in line_start)
        labels[idx] = int(line_no in vuln_lines)

    enc.pop("offset_mapping")
    enc["labels"] = labels.tolist()
    return enc

cols = list(train_df.columns)                         # drop raw cols afterwards
train_tok = Dataset.from_pandas(train_df).map(add_token_labels,
                                              remove_columns=cols)
val_tok   = Dataset.from_pandas(val_df  ).map(add_token_labels,
                                              remove_columns=cols)
test_tok  = Dataset.from_pandas(test_df ).map(add_token_labels,
                                              remove_columns=cols)

# --------------------------------------------------------------------------- #
# 7-B.  model + trainer                                                       #
# --------------------------------------------------------------------------- #
tok_model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=2).cuda()

tok_collator = DataCollatorForTokenClassification(tokenizer=tok, padding="longest")

args_tok = TrainingArguments(
    "line-results",
    fp16=True,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    learning_rate=3e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",
    seed=seed,
    report_to="none",
)

def tok_metrics(eval_pred):
    logits, y = eval_pred
    p = logits.argmax(-1).flatten()
    y = y.flatten()
    mask = y != -100                         # ignore padding tokens
    p, y = p[mask], y[mask]
    prec, rec, f1, _ = precision_recall_fscore_support(
        y, p, average="binary", zero_division=0)
    return {"precision": prec, "recall": rec, "f1": f1}

tok_trainer = Trainer(
    model=tok_model,
    args=args_tok,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tok,
    data_collator=tok_collator,              # NEW collator that pads labels
    compute_metrics=tok_metrics,
)

print("\n===  M3  training (token / line level)  ===")
tok_trainer.train()

print("\n===  Dev metrics (line level) ===")
print(tok_trainer.evaluate(val_tok))


Map:   0%|          | 0/142499 [00:00<?, ? examples/s]

Map:   0%|          | 0/15834 [00:00<?, ? examples/s]

Map:   0%|          | 0/27942 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===  M3  training (token / line level)  ===


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.047400,0.037038,0.991676,0.115427,0.206785
2,0.037700,0.031706,0.754479,0.290526,0.419511
3,0.027600,0.030579,0.671139,0.437811,0.529928



===  Dev metrics (line level) ===


{'eval_loss': 0.030578965321183205, 'eval_precision': 0.6711386642127185, 'eval_recall': 0.437810683292611, 'eval_f1': 0.5299280401292601, 'eval_runtime': 42.1169, 'eval_samples_per_second': 375.954, 'eval_steps_per_second': 5.888, 'epoch': 3.0}


In [10]:
tok_trainer.save_model("line-classifier")
tok.save_pretrained("line-classifier")

('line-classifier/tokenizer_config.json',
 'line-classifier/special_tokens_map.json',
 'line-classifier/vocab.json',
 'line-classifier/merges.txt',
 'line-classifier/added_tokens.json',
 'line-classifier/tokenizer.json')

In [11]:
###############################################################################
#  8.  UTILITY: predict vulnerable lines in one function                      #
###############################################################################
def vulnerable_lines(code: str):
    # 1️⃣ tokenise WITH offsets (for line lookup later)
    enc = tok(
        code,
        return_offsets_mapping=True,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt",
    )

    offset_mapping = enc.pop("offset_mapping")   #  << drop before model call
    enc = {k: v.to(tok_model.device) for k, v in enc.items()}

    # 2️⃣ forward pass --------------------------------------------------------
    with torch.no_grad():
        pred_ids = tok_model(**enc).logits.argmax(-1)[0].cpu()

    # 3️⃣ convert token-level tags → line numbers ----------------------------
    vuln = set()
    for tag, (start, _) in zip(pred_ids, offset_mapping[0]):
        if tag == 1:
            vuln.add(code[:start].count("\n") + 1)      # 1-based lines
    return vuln


# Demo ------------------------------------------------------------------------
demo_code = df.query("label == 1").code.iloc[0]
print("Predicted vulnerable lines:", vulnerable_lines(demo_code))


Predicted vulnerable lines: set()


In [12]:
import pygments, pygments.lexers, pygments.formatters
from termcolor import colored

def demo_vuln(idx=0):
    """
    idx – row index in df (use a query, random sample, …)
    Prints code with GT   lines in  GREEN  ▲
                   and     predicted lines in  RED   ●
    """
    row = df.iloc[idx]
    code = row.code
    gt_lines = set(row.flaw_line_no)          # ground truth
    pred_lines = vulnerable_lines(code)       # our model

    print(colored(f"\nBigVul-ID: {row.bigvul_id}", "yellow"))
    print(colored(f"GT vulnerable lines      : {sorted(gt_lines)}", "green"))
    print(colored(f"Predicted vulnerable lines: {sorted(pred_lines)}", "red"))

    # ► pretty print the function with colouring
    lexer = pygments.lexers.CLexer()
    formatter = pygments.formatters.TerminalFormatter()
    for i, line in enumerate(code.splitlines(), 1):
        marker = ""
        if i in gt_lines:   marker += colored("▲", "green")
        if i in pred_lines: marker += colored("●", "red")
        coloured_code = pygments.highlight(line, lexer, formatter).strip()
        print(f"{i:>3} {marker:<2} {coloured_code}")

# --- run on a few vulnerable functions until you see matches  --------------
for idx in df.query("label==1").sample(5, random_state=0).index:
    demo_vuln(idx)


BigVul-ID: 186441
GT vulnerable lines      : [4, 7, 8, 9]
Predicted vulnerable lines: []
  1     void FrameFetchContext::DispatchDidBlockRequest(
  2         const ResourceRequest& resource_request,
  3         const FetchInitiatorInfo& fetch_initiator_info,
  4 ▲      ResourceRequestBlockedReason blocked_reason) const {
  5       if (IsDetached())
  6         return;
  7 ▲     probe::didBlockRequest(GetFrame()->GetDocument(), resource_request,
  8 ▲                            MasterDocumentLoader(), fetch_initiator_info,
  9 ▲                           blocked_reason);
 10     }

BigVul-ID: 181262
GT vulnerable lines      : [3, 4]
Predicted vulnerable lines: [3]
  1     void grubfs_free (GrubFS *gf) {
  2       if (gf) {
  3 ▲●     if (gf->file && gf->file->device)
  4 ▲         free (gf->file->device->disk);
  5         free (gf->file);
  6         free (gf);
  7      }
  8    }

BigVul-ID: 187971
GT vulnerable lines      : [7, 9, 10, 12, 14]
Predicted vulnerable lines: []
  1    vo